In [6]:
import json

def process_chat_record(chat_list):
    roles = {m["role"]: m["content"] for m in chat_list}
    prompt = (
      "<|im_start|>system\n"    + roles.get("system","")    + "<|im_end|>\n"
      "<|im_start|>user\n"      + roles.get("user","")      + "<|im_end|>\n"
      "<|im_start|>assistant\n"
    )
    response = roles.get("assistant","")
    return {"prompt": prompt, "response": response}

with open("Business.jsonl","r",encoding="utf-8") as fin, \
     open("processed_data.jsonl","w",encoding="utf-8") as fout:

    for line in fin:
        line = line.strip()
        if not line: 
            continue
        record    = json.loads(line)                # e.g. {"messages":[..., ..., ...]}
        turns     = record.get("messages", [])       # extract the list of role/content dicts
        if len(turns) < 2:                          # you can decide how to handle partial data
            continue
        entry     = process_chat_record(turns)
        fout.write(json.dumps(entry, ensure_ascii=False) + "\n")


In [2]:
!pip install datasets

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.3/484.3 kB 11.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 9.5 MB/s eta 0:00:00:00:0100:01
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 9.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.1
    Uninstalling tqdm-4.66.1:
      Successfully uninstalled tqdm-4.66.1
  Attempting uninstall: requests
    Found existing installa

In [3]:
import json
from datasets import Dataset

# Step 1: read & validate line by line
records = []
with open("processed_data.jsonl", "r", encoding="utf-8") as f:
    for i, line in enumerate(f, start=1):
        line = line.strip()
        if not line:
            continue
        try:
            records.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"🚨 JSON error on line {i}: {e}")
            print("Contents:", line[:200])
            raise

# Step 2: convert list of dicts into a HF Dataset
train_data = Dataset.from_list(records)

# now you can proceed with your finetuning logic:
# e.g. tokenizer(...), DataCollator, Trainer(..., train_dataset=train_data)
print(f"✅ Loaded {len(train_data)} examples.")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Loaded 26453 examples.
